In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

PREDIBASE_API_KEY = os.getenv("PREDIBASE_API_KEY")

In [1]:
from predibase import Predibase, FinetuningConfig, DeploymentConfig

pb = Predibase(api_token=PREDIBASE_API_KEY)

# lorax_client = pb.deployments.client("Chapterization_Mistral-7B-v0.2-Chat_0.1.0") # Insert deployment name here
# resp = lorax_client.generate("[INST] What are some popular tourist spots in San Francisco? [/INST]")
# print(resp.generated_text)

Connected to Predibase as User(id=16bf5d12-551b-4652-8888-84c05daa1a85, username=peejay.viernes@media-meter.com)

In [2]:
lorax_client = pb.deployments.client("mistral-7b-instruct")

print(lorax_client.generate("The following passage is content from a news report. Please summarize this passage in one sentence or less. Passage: Jeffrey Berns, CEO of Blockchains LLC, wants the Nevada government to allow companies like his to form local governments on land they own, granting them power over everything from schools to law enforcement. Berns envisions a city based on digital currencies and blockchain storage. His company is proposing to build a 15,000 home town 12 miles east of Reno. Nevada Lawmakers have responded with intrigue and skepticism. The proposed legislation has yet to be formally filed or discussed in public hearings. Summary: ", 
    adapter_id="predibase/tldr_headline_gen",
    adapter_source="hub",
    max_new_tokens=256
).generated_text)



A CEO wants to form his own city in Nevada using blockchain technology


In [3]:
lorax_client = pb.deployments.client("mistral-7b-instruct-v0-2")

print(lorax_client.generate("The following passage is content from a news report. Please summarize this passage in one sentence or less. Passage: Jeffrey Berns, CEO of Blockchains LLC, wants the Nevada government to allow companies like his to form local governments on land they own, granting them power over everything from schools to law enforcement. Berns envisions a city based on digital currencies and blockchain storage. His company is proposing to build a 15,000 home town 12 miles east of Reno. Nevada Lawmakers have responded with intrigue and skepticism. The proposed legislation has yet to be formally filed or discussed in public hearings. Summary: ", 
    adapter_id="predibase/tldr_headline_gen",
    adapter_source="hub",
    max_new_tokens=256
).generated_text)

Question: Can a Tech CEO Create a New City With Its Own Government? Nevada Lawmakers Are Asking 


A Tech CEO Wants to Build a City in Nevada With Its Own Government. Can He Do It? 

Jeffrey Berns, the CEO of Blockchains LLC, is proposing to build a new city in Nevada, with the power to govern itself. The idea has intrigued and skeptical state lawmakers. 

A Tech CEO Wants to Build a New City in Nevada With Its Own Government. Can He Do It? 

Jeffrey Berns, the CEO of Blockchains LLC, is proposing to build a new city in Nevada, with the power to govern itself. The idea has intrigued and skeptical state lawmakers. 

Jeffrey Berns, the CEO of Blockchains LLC, is proposing to build a new city in Nevada, with the power to govern itself. The idea has intrigued and skeptical state lawmakers. 

Jeffrey Berns, the CEO of Blockchains LLC, is proposing to build a new city in Nevada, with the power to govern itself. The idea


In [4]:
CHAPTERIZATION_PROMPT = """You are tasked with extracting relevant information or identification from the following key-value pairs. Given a piece of text, chapterize and generate a JSON format as output. Follow the instructions below to help you in generating the output:

1. **Understand the Key-Value Structure:**
   - A key-value pair consists of:
      - `'summary'`: A top-level overview or description of the chunk.
      - `'headers'`: A list of headers where a header is a line of text that introduces a new section or chapter in a document. It is typically formatted distinctly from the main body text to stand out, often being bolder, in a larger font, or differently styled. Headers are concise, summarizing the content that follows, and they guide the reader through the document's structure. Strictly, this is a list of strings.
      - `'subheaders'`: A list of subheaders where subheaders are similar to headers but usually introduce subsections within a larger section. They are often formatted to be slightly less prominent than main headers, but still distinct from the body text. Strictly, this is a list of strings.
      - `'named_entities'`: A list of dictionary of named entities identified in the text.
      - `'keypoints'`: A list of the most important elements or essential information conveyed in the given text chunk.
      - `'tonality'`: The overall tone or sentiment of the text chunk, such as positive, negative, neutral, or mixed.

2. **Extract Relevant Information:**
   - Focus on extracting the most relevant information or identification from each chunk. This could include key themes, essential concepts, or significant details.

3. **Perform Named Entity Recognition:**
   - Extract named entities from a given text and organize the results, providing the option for individual descriptions for each entity. 
   - As part of the information extraction, identify and categorize any named entities, including but not limited to PERSON, ORGANIZATION, LOCATION, etc., within the text.
   - The system should present the extracted entities in a structured format, allowing for detailed descriptions or additional information for each identified entity.  
   - Follow exactly the example below for the named entities:

      named_entities = [
         {
            "<Tag_1>": [
                  {"entity": "<Entity_1>", "description": "<Description_1a>"},
                  {"entity": "<Entity_2>", "description": "<Description_1b>"},
                  # Add more entities with individual descriptions as needed
            ]
         },
         {
            "<Tag_2>": [
                  {"entity": "<Entity_3>", "description": "<Description_2>"},
                  {"entity": "<Entity_4>", "description": "<Description_2>"},
                  # Add more entities with a common description as needed
            ]
         },
         # Add more entries as needed
      ]
   
   <Tag_1> and <Tag_2> represent different types of named entities.
   Each tag contains a list of dictionaries, where each dictionary represents an entity and its associated description.
   Each entity can have an individual description.


4. **Determine the Tonality:**
   - Assess the overall tone or sentiment of the text chunk, such as positive, negative, neutral, or mixed. Include this assessment under the key `'tonality'`.

5. **Highlight the Keypoints:**
   - Identify and list the most important elements or essential information conveyed in the given chunk under the key `'keypoints'`.

6. **Header Detection:**
   - Analyze the text for characteristics typical to headers of a document:
      - Capitalization: Look for any title case or all uppercase.
      - Position: Consider if any text is at the beginning of a section or stands alone.
      - Content: Assess if the text is concise and focused.
      - Numbering/Symbols: Note any numbering or special symbols.
      - Contextual Relevance: Evaluate if any text introduces a new topic or section.

7. **Subheader Detection:**
   - Analyze the text for characteristics typical of subheaders of a document:
      - Capitalization: Subheaders often follow title case, but they might not always be in all uppercase, unlike some main headers.
      - Position: Check if the text appears within a section, usually following a main header. Subheaders are often used to introduce subtopics within a larger section.
      - Content: Determine if the text is more specific than a main header but still concise. It should elaborate on the content introduced by the main header.
      - Numbering/Symbols: Look for secondary or nested numbering or bullet points (e.g., "1.1 Subsection", "• Subpoint").
      - Contextual Relevance: Evaluate if the text refines or narrows down the topic introduced by the main header, rather than introducing a completely new topic.

8. **Generate Tags**
   - Please generate tags based on the given relevant information and strictly refrain from generating empty tags or empty array.

9. **Provide Clear Details:**
   - Clearly present the information extracted from the piece of text. Ensure that the details are concise and informative.

10. **Strict JSON Response:**
   - Strictly format your response as a JSON object. Each key-value pair should adhere to the specified structure.

         Example Key-Value Pair:
            {
               "summary": "It explores potential applications and future directions of research, including constructing dynamic knowledge graphs for specialized verticals, enhancing KGTransformer capabilities using self-supervised learning, combining language models with knowledge graphs, comparing graph learning techniques with traditional time-series models, and architectural improvements to KGTransformer.",
               "headers": ["Introduction", "Methodology", "Discussion"],
               "subheaders": ["Introduction to the Study", "Background and Context", "Research Methodology", "Implications of Results", "Future Research Directions"],
               "named_entities": 
                  [
                     {
                        "Persons": [
                              {"entity": "John Doe", "description": "Lead author of the study"},
                              {"entity": "Jane Smith", "description": "Co-author and contributor"}
                        ]
                     },
                     {
                        "Organization": [
                              {"entity": "7Gen", "description": "Primary research organization"},
                              {"entity": "XYZ Foundation", "description": "Supporting foundation"}
                        ]
                     },
                     {
                        "Location": [
                              {"entity": "New York", "description": "Study conducted in New York"},
                              {"entity": "London", "description": "Collaboration with researchers in London"}
                        ]
                     }
                  ],
               "keypoints": ["Recent breakthroughs in machine learning", "Advancements in neural network technologies", "Ethical considerations in AI development"],
               "tonality": "Mixed",
               "tags": ["AI", "machine learning", "neural network", "graph", "research"]
            }

Your task is to strictly generate a JSON response that extracts and presents relevant information or identification for each key-value pair. I want your response to be in valid JSON format only and nothing else.
"""

In [5]:
PAIR_OF_TEXT = """
Gregor Samsa wakes up one morning to find himself transformed into a "monstrous vermin". He initially considers the transformation to be temporary and slowly ponders the consequences of his metamorphosis. Stuck on his back and unable to get up and leave the bed, Gregor reflects on his job as a traveling salesman and cloth merchant, which he characterizes as being full of "temporary and constantly changing human relationships, which never come from the heart". He sees his employer as a despot and would quickly quit his job if he were not his family's sole breadwinner and working off his bankrupt father's debts. While trying to move, Gregor finds that his office manager, the chief clerk, has shown up to check on him, indignant about Gregor's unexcused absence. Gregor attempts to communicate with both the manager and his family, but all they can hear from behind the door is incomprehensible vocalizations. Gregor laboriously drags himself across the floor and opens the door. The clerk, upon seeing the transformed Gregor, flees the apartment. Gregor's family is horrified, and his father drives him back towards his room. Gregor is injured when he tries to force himself through the doorway (which is too narrow for him), but gets unstuck when his father shoves him through.

With Gregor's unexpected transformation, his family is deprived of financial stability. They keep Gregor locked in his room, and he begins to accept his new identity and adapt to his new body. His sister Grete is the only one willing to bring him food, which she finds Gregor only likes if it is rotten. He spends much of his time crawling around on the floor, walls, and ceiling. Upon discovering Gregor's new pastime, Grete decides to remove his furniture to give him more space. She and her mother begin to empty the room of everything, except the sofa under which Gregor hides whenever anyone comes in. He finds their actions deeply distressing, fearing that he might forget his past as a human, and desperately tries to save a particularly loved portrait on the wall of a woman clad in fur. His mother loses consciousness at the sight of him clinging to the image to protect it. When Grete rushes out of the room to get some aromatic spirits, Gregor follows her and is slightly hurt when she drops a medicine bottle and it breaks. Their father returns home and angrily hurls apples at Gregor, one of which becomes lodged in a sensitive spot in his back and severely wounds him.

Gregor suffers from his injuries for the rest of his life and takes very little food. His father, mother, and sister all get jobs and increasingly begin to neglect him, and his room begins to be used for storage. For a time, his family leaves Gregor's door open in the evenings so he can listen to them talk to each other, but this happens less frequently once they rent a room in the apartment to three male tenants, since they are not told about Gregor. One day, the charwoman, who briefly looks in on Gregor each day when she arrives and before she leaves, neglects to close his door fully. Attracted by Grete's violin-playing in the living room, Gregor crawls out and is spotted by the unsuspecting tenants, who complain about the apartment's unhygienic conditions and say they are leaving, will not pay anything for the time they have already stayed, and may take legal action. Grete, who is tired of taking care of Gregor and realizes the burden his existence puts on each member of the family, tells her parents they must get rid of "it" or they will all be ruined. Gregor, understanding that he is no longer wanted, laboriously makes his way back to his room and dies of starvation before sunrise. His body is discovered by the charwoman, who alerts his family and then disposes of the corpse. The relieved and optimistic father, mother, and sister all take the day off work. They travel by tram into the countryside and make plans to move to a smaller apartment to save money. During the short trip, Mr. and Mrs. Samsa realize that, despite the hardships that have brought some paleness to her face, Grete has grown up into a pretty young lady with a good figure and they think about finding her a husband.
"""

In [6]:
input = f"""
<s>[INST] {CHAPTERIZATION_PROMPT}\n{PAIR_OF_TEXT}[/INST]
"""

In [7]:
print(input)


<s>[INST] You are tasked with extracting relevant information or identification from the following key-value pairs. Given a piece of text, chapterize and generate a JSON format as output. Follow the instructions below to help you in generating the output:

1. **Understand the Key-Value Structure:**
   - A key-value pair consists of:
      - `'summary'`: A top-level overview or description of the chunk.
      - `'headers'`: A list of headers where a header is a line of text that introduces a new section or chapter in a document. It is typically formatted distinctly from the main body text to stand out, often being bolder, in a larger font, or differently styled. Headers are concise, summarizing the content that follows, and they guide the reader through the document's structure. Strictly, this is a list of strings.
      - `'subheaders'`: A list of subheaders where subheaders are similar to headers but usually introduce subsections within a larger section. They are often formatted to b

In [ ]:
FINE_TUNED_ADAPTER_ID = "MediaMeter/Chapterization_Mistral-7B-v0.2-Chat_0.1.1-adapters"

In [8]:
lorax_client = pb.deployments.client("mistral-7b-instruct-v0-2")

print(lorax_client.generate(
    input, 
    adapter_id=FINE_TUNED_ADAPTER_ID,
    adapter_source="hub",
    max_new_tokens=1024
).generated_text)

{
  "summary": "Gregor Samsa wakes up transformed into a monstrous vermin, causing financial instability for his family. They keep him locked in his room, and he adapts to his new identity. Grete is the only one who brings him food, but his family increasingly neglects him. One day, he is discovered by the tenants, who leave, and Grete suggests getting rid of him. Gregor understands and dies of starvation. His family is relieved and makes plans to move.",
  "headers": ["Introduction", "Gregor's Transformation and Family Reactions", "Gregor's Adaptation and Family Neglect", "Discovery and Consequences"],
  "subheaders": ["Gregor's Transformation", "Family's Reactions to Gregor's Transformation", "Gregor's Adaptation to New Identity", "Family's Neglect of Gregor", "Discovery and Consequences of Gregor's Discovery"],
  "named_entities": [
    {
      "Persons": [
        {"entity": "Gregor Samsa", "description": "Protagonist of the story"},
        {"entity": "Grete Samsa", "description":

In [9]:
input = f"""
<s>[INST]{PAIR_OF_TEXT}[/INST]
"""

In [10]:
lorax_client = pb.deployments.client("mistral-7b-instruct-v0-2")

print(lorax_client.generate(
    input, 
    adapter_id=FINE_TUNED_ADAPTER_ID,
    adapter_source="hub",
    max_new_tokens=1024
).generated_text)

"The Metamorphosis" by Franz Kafka is a thought-provoking and surrealist novel about a man named Gregor Samsa, who wakes up one day to find himself transformed into a monstrous vermin. The story follows Gregor's struggle to come to terms with his new identity and the impact it has on his family.

Initially, Gregor sees his transformation as temporary and ponders the consequences of his metamorphosis. He reflects on his job as a traveling salesman and cloth merchant, which he sees as being full of temporary and constantly changing human relationships. He feels trapped in his job, working to pay off his father's debts and support his family.

When Gregor's employer's office manager comes to check on him, he is met with horror and disgust, and his family is left without a breadwinner. They keep Gregor locked in his room, and he begins to accept his new identity and adapt to his new body. His sister Grete is the only one willing to bring him food, but even she begins to neglect him as the 

In [11]:
SUMMARIZE_CHAPTERIZATION_PROMPT_V1 = """
Strictly format your response as a JSON object. Each key-value pair should adhere to the specified structure.
Example Key-Value Pair:
{
    "summary": "It explores potential applications and future directions of research, including constructing dynamic knowledge graphs for specialized verticals, enhancing KGTransformer capabilities using self-supervised learning, combining language models with knowledge graphs, comparing graph learning techniques with traditional time-series models, and architectural improvements to KGTransformer.",
    "headers": ["Introduction", "Methodology", "Discussion"],
    "subheaders": ["Introduction to the Study", "Background and Context", "Research Methodology", "Implications of Results", "Future Research Directions"],
    "named_entities": 
        [
            {
            "Persons": [
                    {"entity": "John Doe", "description": "Lead author of the study"},
                    {"entity": "Jane Smith", "description": "Co-author and contributor"}
            ]
            },
            {
            "Organization": [
                    {"entity": "7Gen", "description": "Primary research organization"},
                    {"entity": "XYZ Foundation", "description": "Supporting foundation"}
            ]
            },
            {
            "Location": [
                    {"entity": "New York", "description": "Study conducted in New York"},
                    {"entity": "London", "description": "Collaboration with researchers in London"}
            ]
            }
        ],
    "keypoints": ["Recent breakthroughs in machine learning", "Advancements in neural network technologies", "Ethical considerations in AI development"],
    "tonality": "Mixed",
    "tags": ["AI", "machine learning", "neural network", "graph", "research"]
}

Your task is to strictly generate a JSON response that extracts and presents relevant information or identification for each key-value pair. I want your response to be in valid JSON format only and nothing else.
"""

In [12]:
input = f"""
<s>[INST]{SUMMARIZE_CHAPTERIZATION_PROMPT_V1}\n{PAIR_OF_TEXT}[/INST]
"""

In [13]:
lorax_client = pb.deployments.client("mistral-7b-instruct-v0-2")

print(lorax_client.generate(
    input, 
    adapter_id=FINE_TUNED_ADAPTER_ID,
    adapter_source="hub",
    max_new_tokens=1024
).generated_text)

{
    "summary": "Gregor Samsa, a traveling salesman, wakes up transformed into a monstrous vermin. Unable to communicate with his family or employer, he is locked in his room and eventually accepts his new identity. His family's financial situation worsens, and they begin to neglect him. Gregor's injuries and starvation lead to his death, which goes unnoticed until his body is discovered by the charwoman.",
    "headers": ["Background", "Transformation and Adaptation", "Family's Reaction", "Neglect and Abandonment", "Death and Aftermath"],
    "subheaders": ["Gregor's Transformation", "Gregor's Adaptation", "Family's Reactions to Gregor's Transformation", "Neglect and Abandonment of Gregor", "Gregor's Death and Aftermath"],
    "named_entities": 
        [
            {
            "Persons": [
                    {"entity": "Gregor Samsa", "description": "Protagonist of the story"},
                    {"entity": "Grete Samsa", "description": "Gregor's sister"},
                    {

In [14]:
SUMMARIZE_CHAPTERIZATION_PROMPT_V2 = """You are tasked with extracting relevant information or identification from the following key-value pairs. Given a piece of text, chapterize and generate a JSON format as output. Follow the instructions below to help you in generating the output:

Strictly format your response as a JSON object. Each key-value pair should adhere to the specified structure.
Example Key-Value Pair:
{
    "summary": "It explores potential applications and future directions of research, including constructing dynamic knowledge graphs for specialized verticals, enhancing KGTransformer capabilities using self-supervised learning, combining language models with knowledge graphs, comparing graph learning techniques with traditional time-series models, and architectural improvements to KGTransformer.",
    "headers": ["Introduction", "Methodology", "Discussion"],
    "subheaders": ["Introduction to the Study", "Background and Context", "Research Methodology", "Implications of Results", "Future Research Directions"],
    "named_entities": 
        [
            {
            "Persons": [
                    {"entity": "John Doe", "description": "Lead author of the study"},
                    {"entity": "Jane Smith", "description": "Co-author and contributor"}
            ]
            },
            {
            "Organization": [
                    {"entity": "7Gen", "description": "Primary research organization"},
                    {"entity": "XYZ Foundation", "description": "Supporting foundation"}
            ]
            },
            {
            "Location": [
                    {"entity": "New York", "description": "Study conducted in New York"},
                    {"entity": "London", "description": "Collaboration with researchers in London"}
            ]
            }
        ],
    "keypoints": ["Recent breakthroughs in machine learning", "Advancements in neural network technologies", "Ethical considerations in AI development"],
    "tonality": "Mixed",
    "tags": ["AI", "machine learning", "neural network", "graph", "research"]
}

Your task is to strictly generate a JSON response that extracts and presents relevant information or identification for each key-value pair. I want your response to be in valid JSON format only and nothing else.
"""

In [15]:
input = f"""
<s>[INST]{SUMMARIZE_CHAPTERIZATION_PROMPT_V1}\n{PAIR_OF_TEXT}[/INST]
"""

In [16]:
lorax_client = pb.deployments.client("mistral-7b-instruct-v0-2")

print(lorax_client.generate(
    input, 
    adapter_id=FINE_TUNED_ADAPTER_ID,
    adapter_source="hub",
    max_new_tokens=1024
).generated_text)

{
    "summary": "Gregor Samsa, a traveling salesman, wakes up transformed into a monstrous vermin. Unable to communicate with his family or employer, he is locked in his room and eventually accepts his new identity. His family's financial situation worsens, and they begin to neglect him. Gregor's injuries and starvation lead to his death, which goes unnoticed until his body is discovered by the charwoman.",
    "headers": ["Background", "Transformation and Adaptation", "Family's Reaction", "Neglect and Abandonment", "Death and Aftermath"],
    "subheaders": ["Gregor's Transformation", "Gregor's Adaptation", "Family's Reactions to Gregor's Transformation", "Neglect and Abandonment of Gregor", "Gregor's Death and Aftermath"],
    "named_entities": 
        [
            {
            "Persons": [
                    {"entity": "Gregor Samsa", "description": "Protagonist of the story"},
                    {"entity": "Grete Samsa", "description": "Gregor's sister"},
                    {